In [5]:
from scipy.io.wavfile import read
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal, stats
import pytest
import math
import ipytest
import cProfile
from cmath import *
ipytest.config(rewrite_asserts=True, magics=True)
__filename__ = "XC"

/Users/kitty/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
def stereo_to_mono(audio):
    return audio.sum(axis=1) / 2

In [ ]:
frq, raw_sample = read("oblivion.wav")
original = stereo_to_mono(raw_sample)

In [ ]:
frq, raw_sample = read("oblivion.cut.wav")
sample = stereo_to_mono(raw_sample)

In [ ]:
noise = np.random.normal(0, 1, sample.shape)
len(noise)

In [ ]:
original_l, = plt.plot(original, label="original")
sample_l, = plt.plot(sample + noise, label="noisey sample")
plt.legend([original_l, sample_l], ["original", "noisey"])
plt.show()

In [6]:
class AudioClip(object):
    def __init__(self, data, frq):
        self.data = data
        self.frq = frq
    
    def low_pass(self, cutoff_frq, nyq_freq, order=4):
        normal_cutoff = float(cutoff_frq) / nyq_freq
        b, a = signal.butter(order, normal_cutoff, btype='lowpass', output='ba')
        y = signal.filtfilt(b, a, data)
        self.data = y
    
    def filter_and_downsample(self, ds_factor=4):
        self.data = signal.decimate(self.data, ds_factor, n=4)
        self.frq = self.frq / ds_factor
    
#     def create_window(self, bin_num, total_width, seconds=0.1):
#         bin_size = int(self.frq * seconds)
#         frame = np.hanning(bin_size)
#         left_pad = np.zeros(bin_num * bin_size)
#         right_pad = np.zeros(total_width - ((bin_num + 1) * bin_size))
#         window = np.concatenate((left_pad, frame, right_pad), axis=0)
#         return window
    
    def apply_window(self, bin_num):
        window = self.create_window(bin_num, len(data))
        return self.data * window
    
    def zero_pad(self, records):
        x = len(records)
        zeroes = pow(2, math.ceil(math.log(x) / math.log(2))) - x
        return np.concatenate((records, [0] * zeroes))
    
    def get_samples(self, bin_num):
        np.hanning(bin_size)
    
    def create_frequencies_array(self, bin_seconds=0.1):
        bin_size = int(frq * bin_seconds)
        num_bins = len(original) // bin_size
        results = []
        for bin_num in range(num_bins): # num_bins
            # values = apply_window(data, frq, bin_num)
            # padded_values = zero_pad(values)
            # padding gives you better resolution, but too expensive
            samples = self.get_samples(bin_num)
            magnitude = [abs(x) for x in np.fft.fft(samples)]
            results.append(magnitude)
            print(bin_num)
        return results
    
    def __str__(self):
        return "Size " + len(self.data) + " sampled at " + self.frq + "Hz"

In [ ]:
result = zero_pad(apply_window(original, frq, 0))
plt.plot(result)
plt.show()

In [ ]:
# Frequency went from 44.1kHz to 11.025kHz
filtered_data.shape

In [ ]:
plt.plot(filtered_data)
plt.show()

In [ ]:
results = create_frequencies(filtered_data)

In [ ]:
plt.plot(results)
plt.show()